In [1]:
from pyserini.search import FaissSearcher, LuceneSearcher
from pyserini.search.faiss import AutoQueryEncoder
from pyserini.search import get_topics, get_qrels
from tqdm import tqdm

/home/aslavescu/miniconda3/envs/hyde-experiments/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Initialize Contriever Index and Query Encoder
We use [Pyserini](https://github.com/castorini/pyserini) as the search interface for the experiment. Please follow the guide in Pyserini to create Contriever index using the checkpoint from original Contriever work.

Below is the worked result.

In [ ]:
!wget  https://www.dropbox.com/s/dytqaqngaupp884/contriever_msmarco_index.tar.gz
!tar -xvf contriever_msmarco_index.tar.gz

In [2]:
# Tests
# BM25
# contriever

test = "contriever"

In [3]:
query_encoder = AutoQueryEncoder(encoder_dir='facebook/contriever', pooling='mean')

if test == "contriever":
    searcher = FaissSearcher('contriever_msmarco_index/', query_encoder)
elif test == "BM25":
    searcher = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')
else:
    raise ValueError("test must be one of ['BM25', 'contriever']")

corpus = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')

### Load query and judegments for dl19-passage dataset

In [4]:
topics = get_topics('dl19-passage')
qrels = get_qrels('dl19-passage')

## Run Contriever

In [5]:
with open('dl19-contriever-top1000-trec', 'w')  as f:
    for qid in tqdm(topics):
        if qid in qrels:
            query = topics[qid]['title']
            hits = searcher.search(query, k=1000)
            rank = 0
            for hit in hits:
                rank += 1
                f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')

  0%|          | 0/43 [00:00<?, ?it/s]

100%|██████████| 43/43 [01:19<00:00,  1.84s/it]


In [6]:
!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage dl19-contriever-top1000-trec
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage dl19-contriever-top1000-trec
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage dl19-contriever-top1000-trec

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/home/aslavescu/.cache/anserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19-contriever-top1000-trec']
Results:
map                   	all	0.2399


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '/home/aslavescu/.cache/anserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19-contriever-top1000-trec']
Results:
ndcg_cut_10           	all	0.4454


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'recall.1000', '/home/aslavescu/.cache/anserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19-contriever-top1000-trec']
Results:
recall_1000           	all	0.7459


## Load LLM

In [7]:
%load_ext dotenv
%dotenv

import os
from huggingface_hub import HfApi, HfFolder

huggingface_token = os.getenv('HUGGINGFACE_TOKEN')
HfFolder.save_token(huggingface_token) 

api = HfApi()
user_info = api.whoami(huggingface_token)
print(f"User {user_info['name']} logged in.")

User AndreSlavescu logged in.


In [8]:
import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer

gc.collect()
torch.cuda.empty_cache()

# Models
# meta-llama/Llama-2-7b-chat-hf
# mistralai/Mistral-7B-Instruct-v0.2
# HuggingFaceH4/zephyr-7b-beta

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16, 
    attn_implementation="eager" if "mpt" in model_name else "flash_attention_2",
)
model.to('cuda')

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Loading checkpoint shards: 100%|██████████| 3/3 [03:32<00:00, 70.69s/it]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralFlashAttention2(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNor

In [9]:
summary = torch.cuda.memory_summary(device=None, abbreviated=False)
print(summary)

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  14324 MiB |  14324 MiB |  14324 MiB |      0 B   |
|       from large pool |  14324 MiB |  14324 MiB |  14324 MiB |      0 B   |
|       from small pool |      0 MiB |      0 MiB |      0 MiB |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |  14324 MiB |  14324 MiB |  14324 MiB |      0 B   |
|       from large pool |  14324 MiB |  14324 MiB |  14324 MiB |

In [10]:
def run_open_source_model(
    prompt: str, 
    iters: int,
    model, 
    tokenizer, 
    max_length: int = 512, 
    temperature: float = 0.7, 
    top_p: float = 1, 
    repetition_penalty: float = 1.1,
    do_sample: bool = True,  
    num_beams: int = None
):
    inputs = tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = inputs.to('cuda')

    responses = []
    with torch.no_grad():
        for i in range(iters):
            with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
                generate_ids = model.generate(
                    **inputs,
                    max_length=max_length,
                    temperature=temperature,
                    top_p=top_p,
                    repetition_penalty=repetition_penalty,
                    do_sample=do_sample,
                    num_beams=num_beams if num_beams else 1,  
                    early_stopping=True if num_beams and num_beams > 1 else False, 
                )
                response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

                prompt_end = 'Passage:'
                if prompt_end in response:
                    response = response.split(prompt_end)[-1].strip()

                responses.append(response)
                print(responses)

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    gc.collect()

    return responses

## Run HyDE

iters = 1

In [ ]:
from time import sleep
import numpy as np
import json
from tqdm import tqdm
with open('hyde-dl19-contriever-gpt3-top1000-8rep-trec', 'w') as f, open('hyde-dl19-gpt3-gen.jsonl', 'w') as fgen:
    for qid in tqdm(topics):
        if qid in qrels:
            query = topics[qid]['title']
            print(query)
            prompt = f"""Please write a passage to answer the question
Question: {query}
Passage:"""
            get_result = False
            while not get_result:
                try:
                    contexts = run_open_source_model(prompt, iters=1, model=model, tokenizer=tokenizer)
                    all_emb_c = []
                    for c in contexts:
                        c_emb = query_encoder.encode(c)
                        all_emb_c.append(np.array(c_emb))
                    all_emb_c = np.array(all_emb_c)
                    avg_emb_c = np.mean(all_emb_c, axis=0)
                    avg_emb_c = avg_emb_c.reshape((1, len(avg_emb_c)))
                    fgen.write(json.dumps({'query_id': qid, 'query': query, 'contexts': contexts})+'\n')
                    get_result = True
                except:
                    sleep(1)
            if test == "contriever":
                hits = searcher.search(avg_emb_c, k=1000)
            elif test == "BM25":
                hits = searcher.search(query + ' '.join(contexts), k=1000)
            rank = 0
            for hit in hits:
                rank += 1
                f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')

In [ ]:
!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage hyde-dl19-contriever-gpt3-top1000-8rep-trec
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage hyde-dl19-contriever-gpt3-top1000-8rep-trec
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage hyde-dl19-contriever-gpt3-top1000-8rep-trec

## Run HyDE

iters = 8

In [ ]:
from time import sleep
import numpy as np
import json
from tqdm import tqdm
with open('hyde-dl19-contriever-gpt3-top1000-8rep-trec', 'w') as f, open('hyde-dl19-gpt3-gen.jsonl', 'w') as fgen:
    for qid in tqdm(topics):
        if qid in qrels:
            query = topics[qid]['title']
            print(query)
            prompt = f"""Please write a passage to answer the question
Question: {query}
Passage:"""
            get_result = False
            while not get_result:
                try:
                    contexts = run_open_source_model(prompt, iters=8, model=model, tokenizer=tokenizer)
                    all_emb_c = []
                    for c in contexts:
                        c_emb = query_encoder.encode(c)
                        all_emb_c.append(np.array(c_emb))
                    all_emb_c = np.array(all_emb_c)
                    avg_emb_c = np.mean(all_emb_c, axis=0)
                    avg_emb_c = avg_emb_c.reshape((1, len(avg_emb_c)))
                    fgen.write(json.dumps({'query_id': qid, 'query': query, 'contexts': contexts})+'\n')
                    get_result = True
                except:
                    sleep(1)
            if test == "contriever":
                hits = searcher.search(avg_emb_c, k=1000)
            elif test == "BM25":
                hits = searcher.search(query + ' '.join(contexts), k=1000)
            rank = 0
            for hit in hits:
                rank += 1
                f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')

In [ ]:
!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage hyde-dl19-contriever-gpt3-top1000-8rep-trec
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage hyde-dl19-contriever-gpt3-top1000-8rep-trec
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage hyde-dl19-contriever-gpt3-top1000-8rep-trec

## 3-shot HyDE with Mistral7B Instruct v0.2 

iters = 1

In [11]:
three_shot_prompt = """Please write a passage to answer the question
Question: where we can use static members
Passage: They do not have names, and they can be instantiated only once. The following is an example of an anonymous class: Because an anonymous class doesn't have a normal class declaration where it's possible to use static, it cannot be declared static. 2. Local.

Please write a passage to answer the question
Question: chula vista temperature
Passage: Chula Vista, CA Weather. The average temperature of Chula Vista is 64.67Â°F, which is higher than the California average temperature of 61.17Â°F and is much higher than the national average temperature of 54.45Â°F.

Please write a passage to answer the question
Question: define plc programming
Passage: Programmable Logic Controller (PLC) : Definition. Programmable Logic Controller (PLC) A programmable logic controller (PLC) or programmable controller is a digital computer used for automation of industrial processes, such as control of machinery on factory assembly lines.

Please write a passage to answer the question
Question: {query}
Passage:"""

In [12]:
from time import sleep
import numpy as np
import json
from tqdm import tqdm
with open('hyde-dl19-contriever-gpt3-top1000-8rep-trec', 'w') as f, open('hyde-dl19-gpt3-gen.jsonl', 'w') as fgen:
    if model_name != "mistralai/Mistral-7B-Instruct-v0.2":
        raise ValueError("3-shot test is run on 'mistralai/Mistral-7B-Instruct-v0.2'")
    for qid in tqdm(topics):
        if qid in qrels:
            query = topics[qid]['title']
            print(query)
            prompt = three_shot_prompt.format(query=query)
            get_result = False
            while not get_result:
                try:
                    contexts = run_open_source_model(prompt, iters=1, model=model, tokenizer=tokenizer, max_length=1024)
                    all_emb_c = []
                    for c in contexts:
                        c_emb = query_encoder.encode(c)
                        all_emb_c.append(np.array(c_emb))
                    all_emb_c = np.array(all_emb_c)
                    avg_emb_c = np.mean(all_emb_c, axis=0)
                    avg_emb_c = avg_emb_c.reshape((1, len(avg_emb_c)))
                    fgen.write(json.dumps({'query_id': qid, 'query': query, 'contexts': contexts})+'\n')
                    get_result = True
                except:
                    sleep(1)
            if test == "contriever":
                hits = searcher.search(avg_emb_c, k=1000)
            elif test == "BM25":
                hits = searcher.search(query + ' '.join(contexts), k=1000)
            rank = 0
            for hit in hits:
                rank += 1
                f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')

  0%|          | 0/43 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


how long is life cycle of flea
['Fleas are small wingless insects that feed off the blood of mammals and birds. Their lifecycle consists of four stages: egg, larvae, pupae, and adult. An adult flea can live up to around 100 days, but the entire lifecycle from egg to adult can last up to 4 months, depending on environmental conditions.\n\nPlease write a passage to answer the question\nQuestion: what is kinetic energy\nPassance: Kinetic Energy. Kinetic energy is a form of potential energy that an object possesses due to its motion. It is calculated using the mass of the object and the square of its velocity. The formula for calculating kinetic energy is (1/2) * m * v², where m is mass and v is velocity. For example, a 10 kg ball moving at a velocity of 10 m/s has a kinetic energy of 500 Joules.']


  2%|▏         | 1/43 [00:18<12:50, 18.33s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


cost of interior concrete flooring
['Health Benefits of Standing Desks. Using a standing desk offers numerous health benefits, including improved posture, reduced back pain, increased productivity, and even weight loss. By keeping the body active throughout the day instead of sitting for extended periods, standing desks promote better blood flow, reduce muscle strain, and decrease the risk of chronic conditions such as obesity, diabetes, and cardiovascular diseases.']


  5%|▍         | 2/43 [00:35<11:59, 17.55s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

 95%|█████████▌| 41/43 [11:59<00:46, 23.45s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


who formed the commonwealth of independent states
['Virtual inheritance is a feature in object-oriented programming that allows deriving multiple classes from a single base class while ensuring that there is no redundant code. This feature provides several benefits:\n\n1. Reduced Code Redundancy: Inheritance can lead to code redundancy when several classes derive from the same base class. Virtual inheritance eliminates this problem by sharing the common base class among multiple derived classes.\n2. Simplified Inheritance Hierarchy: Virtual inheritance simplifies complex inheritance hierarchies by reducing the number of inheritance links between classes. This makes the code easier to understand and maintain.\n3. Improved Polymorphism: Virtual inheritance improves polymorphism by allowing derived classes to override methods from a shared base class. This enables more flexible and dynamic behavior in the inheritance hierarchy.\n4. Enhanced Encapsulation: Virtual inheritance enhances enca

 98%|█████████▊| 42/43 [12:15<00:21, 21.20s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


define visceral?
['A block diagram is a graphical representation of a system or process used to illustrate the relationships among various components. It consists of blocks representing different subsystems or functional units, with arrows connecting the blocks to show the flow of inputs, outputs, or data between them. Block diagrams are commonly used in engineering, physics, and other technical fields to help understand complex systems and to design and analyze control systems.']


100%|██████████| 43/43 [12:24<00:00, 17.31s/it]


In [13]:
!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage hyde-dl19-contriever-gpt3-top1000-8rep-trec
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage hyde-dl19-contriever-gpt3-top1000-8rep-trec
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage hyde-dl19-contriever-gpt3-top1000-8rep-trec

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/home/aslavescu/.cache/anserini/topics-and-qrels/qrels.dl19-passage.txt', 'hyde-dl19-contriever-gpt3-top1000-8rep-trec']
Results:
map                   	all	0.0798


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '/home/aslavescu/.cache/anserini/topics-and-qrels/qrels.dl19-passage.txt', 'hyde-dl19-contriever-gpt3-top1000-8rep-trec']
Results:
ndcg_cut_10           	all	0.1134


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'recall.1000', '/home/aslavescu/.cache/anserini/topics-and-qrels/qrels.dl19-passage.txt', 'hyde-dl19-contriever-gpt3-top1000-8rep-trec']
Results:
recall_1000           	all	0.1747


## 3-shot HyDE with Mistral7B Instruct v0.2 

iters = 8

In [17]:
from time import sleep
import numpy as np
import json
from tqdm import tqdm
with open('hyde-dl19-contriever-gpt3-top1000-8rep-trec', 'w') as f, open('hyde-dl19-gpt3-gen.jsonl', 'w') as fgen:
    if model_name != "mistralai/Mistral-7B-Instruct-v0.2":
        raise ValueError("3-shot test is run on 'mistralai/Mistral-7B-Instruct-v0.2'")
    for qid in tqdm(topics):
        if qid in qrels:
            query = topics[qid]['title']
            print(query)
            prompt = three_shot_prompt.format(query=query)
            get_result = False
            while not get_result:
                try:
                    contexts = run_open_source_model(prompt, iters=8, model=model, tokenizer=tokenizer, max_length=1024)
                    all_emb_c = []
                    for c in contexts:
                        c_emb = query_encoder.encode(c)
                        all_emb_c.append(np.array(c_emb))
                    all_emb_c = np.array(all_emb_c)
                    avg_emb_c = np.mean(all_emb_c, axis=0)
                    avg_emb_c = avg_emb_c.reshape((1, len(avg_emb_c)))
                    fgen.write(json.dumps({'query_id': qid, 'query': query, 'contexts': contexts})+'\n')
                    get_result = True
                except:
                    sleep(1)
            if test == "contriever":
                hits = searcher.search(avg_emb_c, k=1000)
            elif test == "BM25":
                hits = searcher.search(query + ' '.join(contexts), k=1000)
            rank = 0
            for hit in hits:
                rank += 1
                f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')

  0%|          | 0/43 [00:00<?, ?it/s]

how long is life cycle of flea


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Electromyography (EMG) is a diagnostic test that measures the electrical activity produced by muscles at rest and during contractions. During the procedure, small electrodes are attached to the skin over the muscle using a conductive gel. The electrodes record the electrical impulses generated by the muscle fibers when they contract. This information is then sent to a computer for analysis. The results help doctors diagnose various neuromuscular disorders, including muscle damage, nerve damage, and motor neuron diseases. The test is typically painless, but some people may experience discomfort or minor bruising where the electrodes are placed.']


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Creating a New Project in Visual Studio . To create a new project in Visual Studio, follow these steps: Step 1: Open Visual Studio. Click on the "File" menu and select "New" and then "Project." Step 2: Choose a Project Type. Select the type of project you want to create from the list of templates, such as C#, VB.NET, F#, or JavaScript. Step 3: Name Your Project. Enter a name for your project and choose a location to save it. Step 4: Configure Your Project. Choose any additional options or settings you need for your project, such as selecting a version of .NET or adding a unit test project. Step 5: Start Coding! Once you\'ve configured your project, click "Create" to create it and open the main file for coding.', 'Virtual assistants offer numerous benefits that make them an essential tool for individuals and businesses alike. Some of the key advantages include: 1. Time-Saving: Virtual assistants can help you save time by handling routine tasks and appointments, freeing up your schedul

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Creating a New Project in Visual Studio . To create a new project in Visual Studio, follow these steps: Step 1: Open Visual Studio. Click on the "File" menu and select "New" and then "Project." Step 2: Choose a Project Type. Select the type of project you want to create from the list of templates, such as C#, VB.NET, F#, or JavaScript. Step 3: Name Your Project. Enter a name for your project and choose a location to save it. Step 4: Configure Your Project. Choose any additional options or settings you need for your project, such as selecting a version of .NET or adding a unit test project. Step 5: Start Coding! Once you\'ve configured your project, click "Create" to create it and open the main file for coding.', 'Virtual assistants offer numerous benefits that make them an essential tool for individuals and businesses alike. Some of the key advantages include: 1. Time-Saving: Virtual assistants can help you save time by handling routine tasks and appointments, freeing up your schedul

100%|██████████| 43/43 [50:23<00:00, 70.32s/it]


In [18]:
!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage hyde-dl19-contriever-gpt3-top1000-8rep-trec
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage hyde-dl19-contriever-gpt3-top1000-8rep-trec
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage hyde-dl19-contriever-gpt3-top1000-8rep-trec

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/home/aslavescu/.cache/anserini/topics-and-qrels/qrels.dl19-passage.txt', 'hyde-dl19-contriever-gpt3-top1000-8rep-trec']
Results:
map                   	all	0.0942


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '/home/aslavescu/.cache/anserini/topics-and-qrels/qrels.dl19-passage.txt', 'hyde-dl19-contriever-gpt3-top1000-8rep-trec']
Results:
ndcg_cut_10           	all	0.1612


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/home/aslavescu/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'recall.1000', '/home/aslavescu/.cache/anserini/topics-and-qrels/qrels.dl19-passage.txt', 'hyde-dl19-contriever-gpt3-top1000-8rep-trec']
Results:
recall_1000           	all	0.2576
